<b>Описание/Пошаговая инструкция выполнения домашнего задания:</b><br>
Выберите один из предложенных ниже датасетов (или предложите свой) и попробуйте воспользоваться методами автоматического построения моделей с помощью H2O AutoML для получения решения поставленной задачи. Сравните точностные характеристики обученной метамодели с любой парой моделей "из коробки" (т.е. выбранных вами вручную под конкретную задачу без дополнительной настройки гиперпараметров). Дополнительно сравните результаты с моделью выбранной с помощью библиотеки TPOT.

# Посмотрим на данные

In [4]:
# Сменим рабочую дирректорию для удобства
import os
os.chdir('/home/slawa/HDD/my_scripts/my_otus/ML_advanced/lesson_5/hw')

In [5]:
# библиотеки
import pandas as pd
import numpy as np

In [6]:
# загрузка данных
raw_data = "RetailMart.xlsx"
!wget -r https://github.com/ankislyakov/data/raw/main/RetailMart.xlsx -O $raw_data

ПРЕДУПРЕЖДЕНИЕ: комбинирование параметра -O с -r или -p означает, что весь
загруженные данные будут помещены в один файл.

--2024-03-28 22:46:57--  https://github.com/ankislyakov/data/raw/main/RetailMart.xlsx
Загружен сертификат CA «/etc/ssl/certs/ca-certificates.crt»
Распознаётся github.com (github.com)… 140.82.121.4
Подключение к github.com (github.com)|140.82.121.4|:443... соединение установлено.


HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://raw.githubusercontent.com/ankislyakov/data/main/RetailMart.xlsx [переход]
--2024-03-28 22:46:58--  https://raw.githubusercontent.com/ankislyakov/data/main/RetailMart.xlsx
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 70664 (69K) [application/octet-stream]
Сохранение в: «RetailMart.xlsx»

RetailMart.xlsx     100%[===================>]  69,01K   383KB/s    за 0,2s    

2024-03-28 22:46:59 (383 KB/s) - «RetailMart.xlsx» сохранён [70664/70664]

ЗАВЕРШЕНО --2024-03-28 22:46:59--
Общее время: 1,5s
Загружено: 1 файлов, 69K за 0,2s (383 KB/s)


In [7]:
data = pd.read_excel(raw_data)
data

/home/slawa/HDD/my_scripts/my_otus/ML_advanced/venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Implied Gender,Home/Apt/ PO Box,Pregnancy Test,Birth Control,Feminine Hygiene,Folic Acid,Prenatal Vitamins,Prenatal Yoga,Body Pillow,Ginger Ale,Sea Bands,Stopped buying ciggies,Cigarettes,Smoking Cessation,Stopped buying wine,Wine,Maternity Clothes,Unnamed: 17,PREGNANT
0,M,A,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,1
1,M,H,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,1
2,M,H,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,NaN,1
3,U,H,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,NaN,1
4,F,A,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,M,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0
996,F,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0
997,M,H,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0
998,U,H,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0


In [8]:
print(*data.columns, sep="\n")

Implied Gender
Home/Apt/ PO Box
Pregnancy Test
Birth Control
Feminine Hygiene
Folic Acid
Prenatal Vitamins
Prenatal Yoga
Body Pillow
Ginger Ale
Sea Bands
Stopped buying ciggies
Cigarettes
Smoking Cessation
Stopped buying wine
Wine
Maternity Clothes
Unnamed: 17
PREGNANT


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Implied Gender          1000 non-null   object 
 1   Home/Apt/ PO Box        1000 non-null   object 
 2   Pregnancy Test          1000 non-null   int64  
 3   Birth Control           1000 non-null   int64  
 4   Feminine Hygiene        1000 non-null   int64  
 5   Folic Acid              1000 non-null   int64  
 6   Prenatal Vitamins       1000 non-null   int64  
 7   Prenatal Yoga           1000 non-null   int64  
 8   Body Pillow             1000 non-null   int64  
 9   Ginger Ale              1000 non-null   int64  
 10  Sea Bands               1000 non-null   int64  
 11  Stopped buying ciggies  1000 non-null   int64  
 12  Cigarettes              1000 non-null   int64  
 13  Smoking Cessation       1000 non-null   int64  
 14  Stopped buying wine     1000 non-null   i

In [10]:
data.drop(['Unnamed: 17'], axis=1, inplace=True)

In [11]:
data["Implied Gender"].unique()

array(['M', 'U', 'F'], dtype=object)

In [12]:
data["Home/Apt/ PO Box"].unique()

array(['A', 'H', 'P'], dtype=object)

In [13]:
def bin_encoding(data:pd.core.frame.DataFrame,
                 column_name:str,
                 zero_class:str | None = None) -> pd.core.frame.DataFrame:
    '''
    Кодирование катигориальных признаков в двоичную систему
    '''
    data = data.copy()
    if column_name not in data.columns:
        return data
    classes = data[column_name].unique()
    if zero_class:
        classes = classes[classes != zero_class]
    for num, cl in enumerate(classes):
        new_column_name = column_name + '_' + str(num)
        data[new_column_name] = 0
        data.loc[data[column_name] == cl, new_column_name] = 1
    data.drop([column_name], axis=1, inplace=True)
    return data

def drop_untitled(data:pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    '''
    удаление столбцов с именем Unnamed
    '''
    columns_name = data.columns
    mask = list(map(lambda x: 'Unnamed' not in x, columns_name))
    return data[columns_name[mask]]



In [14]:
drop_untitled(data)

,Implied Gender,Home/Apt/ PO Box,Pregnancy Test,Birth Control,Feminine Hygiene,Folic Acid,Prenatal Vitamins,Prenatal Yoga,Body Pillow,Ginger Ale,Sea Bands,Stopped buying ciggies,Cigarettes,Smoking Cessation,Stopped buying wine,Wine,Maternity Clothes,PREGNANT
0,M,A,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,M,H,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
2,M,H,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
3,U,H,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
4,F,A,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,M,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
996,F,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
997,M,H,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
998,U,H,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Implied Gender          1000 non-null   object
 1   Home/Apt/ PO Box        1000 non-null   object
 2   Pregnancy Test          1000 non-null   int64 
 3   Birth Control           1000 non-null   int64 
 4   Feminine Hygiene        1000 non-null   int64 
 5   Folic Acid              1000 non-null   int64 
 6   Prenatal Vitamins       1000 non-null   int64 
 7   Prenatal Yoga           1000 non-null   int64 
 8   Body Pillow             1000 non-null   int64 
 9   Ginger Ale              1000 non-null   int64 
 10  Sea Bands               1000 non-null   int64 
 11  Stopped buying ciggies  1000 non-null   int64 
 12  Cigarettes              1000 non-null   int64 
 13  Smoking Cessation       1000 non-null   int64 
 14  Stopped buying wine     1000 non-null   int64 
 15  Wine 

In [16]:
data_bin_encoding.describe()

NameError: name 'data_bin_encoding' is not defined

# Собираем pipeline

In [1]:
# Сменим рабочую дирректорию для удобства
import os
os.chdir('/home/slawa/HDD/my_scripts/my_otus/ML_advanced/lesson_5/hw')

In [2]:
# библиотеки
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# функции
def bin_encoding(data:pd.DataFrame,
                 column_name:str | None = None,
                 zero_class:str | None = None) -> pd.DataFrame:
    '''
    Кодирование катигориальных признаков в двоичную систему
    '''
    data = data.copy()
    if column_name not in data.columns:
        return data
    classes = data[column_name].unique()
    if zero_class:
        classes = classes[classes != zero_class]
    for num, cl in enumerate(classes):
        new_column_name = column_name + '_' + str(num)
        data[new_column_name] = 0
        data.loc[data[column_name] == cl, new_column_name] = 1
    data.drop([column_name], axis=1, inplace=True)
    return data

In [ ]:
# загрузка данных
raw_data = "RetailMart.xlsx"
!wget -r https://github.com/ankislyakov/data/raw/main/RetailMart.xlsx -O $raw_data

ПРЕДУПРЕЖДЕНИЕ: комбинирование параметра -O с -r или -p означает, что весь
загруженные данные будут помещены в один файл.

--2024-03-26 23:35:18--  https://github.com/ankislyakov/data/raw/main/RetailMart.xlsx
Загружен сертификат CA «/etc/ssl/certs/ca-certificates.crt»
Распознаётся github.com (github.com)… 140.82.121.3
Подключение к github.com (github.com)|140.82.121.3|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://raw.githubusercontent.com/ankislyakov/data/main/RetailMart.xlsx [переход]
--2024-03-26 23:35:18--  https://raw.githubusercontent.com/ankislyakov/data/main/RetailMart.xlsx
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 70664 (69K) [application/octet-stream]
Сохранение в: «RetailMart.xls

In [ ]:
data = pd.read_excel(raw_data)
data

/home/slawa/HDD/my_scripts/my_otus/ML_advanced/venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Implied Gender,Home/Apt/ PO Box,Pregnancy Test,Birth Control,Feminine Hygiene,Folic Acid,Prenatal Vitamins,Prenatal Yoga,Body Pillow,Ginger Ale,Sea Bands,Stopped buying ciggies,Cigarettes,Smoking Cessation,Stopped buying wine,Wine,Maternity Clothes,Unnamed: 17,PREGNANT
0,M,A,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,1
1,M,H,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,1
2,M,H,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,NaN,1
3,U,H,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,NaN,1
4,F,A,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,M,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0
996,F,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0
997,M,H,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0
998,U,H,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0


In [ ]:
print(*data.columns, sep="\n")

Implied Gender
Home/Apt/ PO Box
Pregnancy Test
Birth Control
Feminine Hygiene
Folic Acid
Prenatal Vitamins
Prenatal Yoga
Body Pillow
Ginger Ale
Sea Bands
Stopped buying ciggies
Cigarettes
Smoking Cessation
Stopped buying wine
Wine
Maternity Clothes
Unnamed: 17
PREGNANT


In [ ]:
target = ['PREGNANT']
categorical = ['Implied Gender', 'Home/Apt/ PO Box']
numeric = [
    'Pregnancy Test',
    'Birth Control',
    'Feminine Hygiene',
    'Folic Acid',
    'Prenatal Vitamins',
    'Prenatal Yoga',
    'Body Pillow',
    'Ginger Ale',
    'Sea Bands',
    'Stopped buying ciggies',
    'Cigarettes',
    'Smoking Cessation',
    'Stopped buying wine',
    'Wine',
    'Maternity Clothes'
]
data_train, data_test = train_test_split(data, test_size=0.2, stratify=data[target])
print(data_train.shape, data_test.shape)

(800, 19) (200, 19)


In [ ]:
categorical_transformer = FunctionTransformer(bin_encoding)

In [ ]:
categorical_transformer = Pipeline(
    steps=[
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ]
)

preprocessor = Pipeline(steps=[("data_transformer", categorical_transformer)])

classifier_pipline = Pipeline(
    steps=[('preprocessor', preprocessor),
           ('classifier', LogisticRegression())])


In [ ]:
classifier_pipline

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('data_transformer',
                                  Pipeline(steps=[('onehot',
                                                   OneHotEncoder(handle_unknown='ignore'))]))])),
                ('classifier', LogisticRegression())])